In [60]:
#creating probabilistic assignments for the anatomical thalamus segmentations 

import Functional_Fusion.atlas_map as am
import HierarchBayesParcel.arrangements as ar
import nibabel as  nb
import numpy as np 
import torch as pt 
import ants 

wk_dir = '/Users/incehusain/fs_projects'

sub = 'sub-01'

segmentation_path = f"{wk_dir}/dseg_segmentations_Language7T/{sub}_thalamus_dseg.nii.gz"

template = f"{wk_dir}/tpl-MNI152NLin2009cSym_res-1_T1w.nii"

lut_file = f"{wk_dir}/compressionLookupTable.txt"

affine_transform_path = f"{wk_dir}/pseg_segmentations_Language7T/{sub}_to_MNI_0GenericAffine.mat"
warp_transform_path = f"{wk_dir}/pseg_segmentations_Language7T/{sub}_to_MNI_1Warp.nii.gz"
transform_list = [affine_transform_path, warp_transform_path]

sub_prob_map_path = f"{wk_dir}/{sub}_thalamus_pseg.nii.gz"

lut_label_to_name = {}
with open(lut_file, 'r') as f:
    for line in f:
        if line.strip() and not line.startswith('#'):
            parts = line.strip().split()
            label = int(parts[0])
            name = parts[1]
            lut_label_to_name[label] = name
thalamus_labels = sorted(list(lut_label_to_name.keys()))

K = len(thalamus_labels)    

template_img = ants.image_read(template)
seg_img = ants.image_read(segmentation_path)

mask_images = [(seg_img == label).astype('float32') for label in thalamus_labels]


seg_to_4d_img = ants.merge_channels(mask_images)

warped_prob_4d_img = ants.apply_transforms(fixed=template_img, moving=seg_to_4d_img, transformlist=transform_list, interpolator='linear')

prob_array_kxp = warped_prob_4d_img.numpy().T

column_sums = prob_array_kxp.sum(axis=0) 
column_sums[column_sums == 0] = 1
normalized_prob_array_kxp = prob_array_kxp / column_sums

normalized_prob_array_kxp.shape

print("TEST")


: 

In [55]:
import ants
import numpy as np
import os

wk_dir = '/Users/incehusain/fs_projects'

sub = 'sub-01'

native_seg_path = f"{wk_dir}/dseg_segmentations_Language7T/{sub}_thalamus_dseg.nii.gz"
template_path = f"{wk_dir}/tpl-MNI152NLin2009cSym_res-1_T1w.nii"
custom_lut_path = f"{wk_dir}/compressionLookupTable.txt"

affine_transform_path = f"{wk_dir}/pseg_segmentations_Language7T/{sub}_to_MNI_0GenericAffine.mat"
warp_transform_path = f"{wk_dir}/pseg_segmentations_Language7T/{sub}_to_MNI_1Warp.nii.gz"
transform_list = [warp_transform_path, affine_transform_path]

final_kxp_path = f"{wk_dir}/{sub}_thalamus_probmap_in_MNI.npy"

lut_label_to_name = {}
with open(lut_file, 'r') as f:
    for line in f:
        if line.strip() and not line.startswith('#'):
            parts = line.strip().split()
            label = int(parts[0])
            name = parts[1]
            lut_label_to_name[label] = name

thalamus_labels = sorted(list(lut_label_to_name.keys()))

K = len(thalamus_labels)    

template_img = ants.image_read(template_path)
native_seg_img = ants.image_read(native_seg_path)
   
seg_np = native_seg_img.numpy()
one_hot_np = np.zeros(seg_np.shape + (K,), dtype='float32')
label_to_index = {label: i for i, label in enumerate(thalamus_labels)}

for i, label in enumerate(thalamus_labels):
    one_hot_np[..., i] = (seg_np == label).astype('float32')

one_hot_native_img = ants.from_numpy(one_hot_np, origin=native_seg_img.origin, spacing=native_seg_img.spacing, direction=native_seg_img.direction)

warped_prob_4d_img = ants.apply_transforms(fixed=template_img,
                                           moving=one_hot_np,
                                           transformlist=transform_list,
                                           interpolator='linear')

prob_array_kp = warped_prob_4d_img.numpy().T
column_sums = prob_array_kp.sum(axis=0)
column_sums[column_sums == 0] = 1.0
final_prob_map = prob_array_kp / column_sums

#np.save(final_kxp_path, final_prob_map)
print(f"Successfully saved the final KxP matrix to: {final_kxp_path}")



ValueError: must give a origin value for each dimension (4)

In [38]:
import ants
import numpy as np
import os

wk_dir = '/Users/incehusain/fs_projects'

sub = 'sub-19'

native_seg_path = f"{wk_dir}/dseg_segmentations_Language7T/{sub}_thalamus_dseg.nii.gz"
template_path = f"{wk_dir}/tpl-MNI152NLin2009cSym_res-1_T1w.nii"
custom_lut_path = f"{wk_dir}/compressionLookupTable.txt"

affine_transform_path = f"{wk_dir}/{sub}_to_MNI_0GenericAffine.mat"
warp_transform_path = f"{wk_dir}/{sub}_to_MNI_1Warp.nii.gz"
transform_list = [warp_transform_path, affine_transform_path]

final_kxp_path = f"{wk_dir}/{sub}_thalamus_probmap_in_MNI.npy"

lut_label_to_name = {}
with open(lut_file, 'r') as f:
    for line in f:
        if line.strip() and not line.startswith('#'):
            parts = line.strip().split()
            label = int(parts[0])
            name = parts[1]
            lut_label_to_name[label] = name

thalamus_labels = sorted(list(lut_label_to_name.keys()))

K = len(thalamus_labels)    

template_img = ants.image_read(template_path)
native_seg_img = ants.image_read(native_seg_path)

#target_pixeltype = native_seg_img.pixeltype 

#mask_images = [(native_seg_img == label).astype('float32') for label in thalamus_labels]

#one_hot_native_img = ants.list_to_ndimage(native_seg_img, mask_images)    

seg_np = native_seg_img.numpy()
one_hot_np = np.zeros(seg_np.shape + (K,), dtype='float32')
label_to_index = {label: i for i, label in enumerate(thalamus_labels)}

for i, label in enumerate(thalamus_labels):
    one_hot_np[..., i] = (seg_np == label).astype('float32')
    
one_hot_native_img = ants.from_numpy(data = one_hot_np,
                                     origin=native_seg_img.origin,
                                     spacing=native_seg_img.spacing,
                                     direction=native_seg_img.direction)

warped_prob_4d_img = ants.apply_transforms(fixed=template_img,
                                           moving=one_hot_native_img,
                                           transformlist=transform_list,
                                           interpolator='linear')

prob_array_kp = warped_prob_4d_img.numpy().T
column_sums = prob_array_kp.sum(axis=0)
column_sums[column_sums == 0] = 1.0
final_prob_map = prob_array_kp / column_sums

#np.save(final_kxp_path, final_prob_map)
print(f"Successfully saved the final KxP matrix to: {final_kxp_path}")



KeyboardInterrupt: 

In [ ]:
#creating KxP hard assignment matrix for the anatomical thalamus segmentations

import Functional_Fusion.atlas_map as am
import HierarchBayesParcel.arrangements as ar
import nibabel as  nb
import numpy as np
import torch as pt

wk_dir = '/Users/incehusain/fs_projects'
iglesias_atlas_nii = f"{wk_dir}/dseg_segmentations_Language7T/sub_19_thalamus_dseg.nii.gz"
lut_file = f"{wk_dir}/compressionLookupTable.txt"
iglesias_head = '# Labels for thalamus parcellation using histological atlas (Iglesias et al.)'
lut_label_to_name = {}

with open(lut_file, 'r') as f:
    for line in f:
        if line.strip() and not line.startswith('#'):
            parts = line.strip().split()
            label = int(parts[0])
            name = parts[1]
            lut_label_to_name[label] = name

thalamus_labels = sorted(list(lut_label_to_name.keys()))

K = len(thalamus_labels)

img = nb.load(iglesias_atlas_nii)

data = img.get_fdata().astype(np.int32)

thalamus_voxel = data > 0

voxel_labels = data[thalamus_voxel]

P=len(voxel_labels)
#we are no longer reading it into MNISymThalamus1 space - going to make new ROI mask using anatomical images
#atlas, _ = am.get_atlas('MNISymThalamus1')
#U = atlas.read_data(iglesias_atlas_nii)
#P=U.shape[0]

label_to_index = {label: idx for idx, label in enumerate(thalamus_labels)}

prob_map_numpy = np.zeros((K, P))

for p, label in enumerate(voxel_labels):
    if label in label_to_index:
        k = label_to_index[label]
        prob_map_numpy[k, p] = 1.0

U_iglesias = np.array(prob_map_numpy)

U_iglesias_map = np.round(U_iglesias)
#ar_model = ar.build_arrangement_model(U_iglesias_map, prior_type='prob', atlas=atlas)
#print("TEST")


In [21]:
import ants 
import os

wk_dir = '/Users/incehusain/fs_projects'

sub = 'sub-03'

output_prefix = f"{wk_dir}/{sub}_to_MNI_"

template = f"{wk_dir}/tpl-MNI152NLin2009cSym_res-1_T1w.nii"
T1_nii = f"{wk_dir}/freesurfer_Language7T/{sub}/mri/T1.mgz"   #or norm.mgz
seg_nii = f"{wk_dir}/dseg_segmentations_Language7T/{sub}_thalamus_dseg.nii.gz"

output_path = f"{wk_dir}"

template_ants = ants.image_read(template)
T1 = ants.image_read(T1_nii)
seg = ants.image_read(seg_nii)

print('Registering...')

registration = ants.registration(fixed=template_ants, moving=T1, type_of_transform='SyN', outprefix=output_prefix)

print('Registration done!')

warped_seg = ants.apply_transforms(fixed=template_ants, moving=seg, transformlist=registration['fwdtransforms'], interpolator='linear')

ants.image_write(warped_seg, f"{wk_dir}/{sub}_seg_warped.nii.gz")



Registering...
Registration done!
